# Azure AI Content Understanding - Classifier and Analyzer Demo

This notebook demonstrates how to use Azure AI Content Understanding service to:
1. Create a classifier to categorize documents
2. Create a custom analyzer to extract specific fields
3. Combine classifier and analyzers to classify, optionally split, and analyze documents in a flexible processing pipeline

If you’d like to learn more before getting started, see the official documentation:
[Understanding Classifiers in Azure AI Services](https://learn.microsoft.com/en-us/azure/ai-services/content-understanding/concepts/classifier)

## Prerequisites
1. Ensure Azure AI service is configured following [steps](../README.md#configure-azure-ai-service-resource)
2. Install the required packages to run the sample.


In [ ]:
%pip install -r ../requirements.txt

## 1. Import Required Libraries

In [ ]:
import json
import logging
import os
import sys
import uuid
from pathlib import Path

from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

print("✅ Libraries imported successfully!")

## 2. Import Azure Content Understanding Client

The `AzureContentUnderstandingClient` class handles all API interactions with the Azure AI service.

In [ ]:
# Add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
try:
    from python.content_understanding_client import AzureContentUnderstandingClient
    print("✅ Azure Content Understanding Client imported successfully!")
except ImportError:
    print("❌ Error: Make sure 'AzureContentUnderstandingClient.py' is in the same directory as this notebook.")
    raise

## 3. Configure Azure AI Service Settings and Prepare the Sample

Update these settings to match your Azure environment:

- **AZURE_AI_ENDPOINT**: Your Azure AI service endpoint URL or set up in ".env" file
- **AZURE_AI_API_VERSION**: The Azure AI API version to use. Default is "2025-05-01-preview". 
- **AZURE_AI_API_KEY**: Your Azure AI service key (optional if using token authentication)
- **ANALYZER_SAMPLE_FILE**: Path to the PDF document you want to process

In [ ]:
# For authentication, you can use either token-based auth or subscription key, and only one of them is required
AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
# IMPORTANT: Replace with your actual subscription key or set up in ".env" file if not using token auth
AZURE_AI_API_KEY = os.getenv("AZURE_AI_API_KEY")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2025-05-01-preview")
ANALYZER_SAMPLE_FILE = "../data/mixed_financial_docs.pdf"  # Update this path to your PDF file

# Authentication - Using DefaultAzureCredential for token-based auth
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

file_location = Path(ANALYZER_SAMPLE_FILE)

print("📋 Configuration Summary:")
print(f"   Endpoint: {AZURE_AI_ENDPOINT}")
print(f"   API Version: {AZURE_AI_API_VERSION}")
print(f"   Document: {file_location.name if file_location.exists() else '❌ File not found'}")

## 4. Define Classifier Schema

The classifier schema defines:
- **Categories**: Document types to classify (e.g., Legal, Medical)
  - **description (Optional)**: An optional field used to provide additional context or hints for categorizing or splitting documents. This can be helpful when the category name alone isn’t descriptive enough. If the category name is already clear and self-explanatory, this field can be omitted.
- **splitMode Options**: Defines how multi-page documents should be split before classification or analysis.
  - `"auto"`: Automatically split based on content.  
  For example, if two categories are defined as “invoice” and “application form”:
    - A PDF with only one invoice will be classified as a single document.
    - A PDF containing two invoices and one application form will be automatically split into three classified sections.
  - `"none"`: No splitting.  
  The entire multi-page document is treated as a single unit for classification and analysis.
  - `"perPage"`: Split by page.  
  Each page is treated as a separate document. This is useful when you’ve built custom analyzers designed to operate on a per-page basis.

In [ ]:
# Define document categories and their descriptions
classifier_schema = {
    "categories": {
        "Loan application": {  # Both space and underscore are allowed in category names
            "description": "Documents submitted by individuals or businesses to request funding, typically including personal or business details, financial history, loan amount, purpose, and supporting documentation."
        },
        "Invoice": {
            "description": "Billing documents issued by sellers or service providers to request payment for goods or services, detailing items, prices, taxes, totals, and payment terms."
        },
        "Bank_Statement": {  # Both space and underscore are allowed in category names
            "description": "Official statements issued by banks that summarize account activity over a period, including deposits, withdrawals, fees, and balances."
        },
    },
    "splitMode": "auto"  # IMPORTANT: Automatically detect document boundaries. Can change mode for your needs.
}

print("📄 Classifier Categories:")
for category, details in classifier_schema["categories"].items():
    print(f"   • {category}: {details['description'][:60]}...")

## 5. Initialize Content Understanding Client

Create the client that will communicate with Azure AI services.

⚠️ Important:
You must update the code below to match your Azure authentication method.
Look for the `# IMPORTANT` comments and modify those sections accordingly.
If you skip this step, the sample may not run correctly.

⚠️ Note: Using a subscription key works, but using a token provider with Azure Active Directory (AAD) is much safer and is highly recommended for production environments.

In [ ]:
# Initialize the Azure Content Understanding client
try:
    content_understanding_client = AzureContentUnderstandingClient(
        endpoint=AZURE_AI_ENDPOINT,
        api_version=AZURE_AI_API_VERSION,
        # IMPORTANT: Comment out token_provider if using subscription key
        token_provider=token_provider,
        # IMPORTANT: Uncomment this if using subscription key
        # subscription_key=AZURE_AI_API_KEY,
    )
    print("✅ Content Understanding client initialized successfully!")
    print("   Ready to create classifiers and analyzers.")
except Exception as e:
    print(f"❌ Failed to initialize client: {e}")
    raise

## 6. Create a Basic Classifier

First, we'll create a simple classifier that categorizes documents without additional analysis.

In [ ]:
# Generate unique classifier ID
classifier_id = "classifier-sample-" + str(uuid.uuid4())

try:
    # Create classifier
    print(f"🔨 Creating classifier: {classifier_id}")
    print("   This may take a few seconds...")
    
    response = content_understanding_client.begin_create_classifier(classifier_id, classifier_schema)
    result = content_understanding_client.poll_result(response)
    
    print("\n✅ Classifier created successfully!")
    print(f"   Status: {result.get('status')}")
    print(f"   Resource Location: {result.get('resourceLocation')}")
    
except Exception as e:
    print(f"\n❌ Error creating classifier: {e}")
    if "already exists" in str(e):
        print("\n💡 Tip: The classifier already exists. You can:")
        print("   1. Use a different classifier ID")
        print("   2. Delete the existing classifier first")
        print("   3. Skip to document classification")

## 7. Classify Your Document

Now let's use the classifier to categorize your document.

In [ ]:
try:
    # Check if document exists
    if not file_location.exists():
        raise FileNotFoundError(f"Document not found at {file_location}")
    
    # Classify document
    print(f"📄 Classifying document: {file_location.name}")
    print("\n⏳ Processing... This may take a few minutes for large documents.")
    
    response = content_understanding_client.begin_classify(classifier_id, file_location=str(file_location))
    result = content_understanding_client.poll_result(response, timeout_seconds=360)
    
    print("\n✅ Classification completed successfully!")
    
except FileNotFoundError:
    print(f"\n❌ Document not found: {file_location}")
    print("   Please update file_location to point to your PDF file.")
except Exception as e:
    print(f"\n❌ Error classifying document: {e}")

## 8. View Classification Results

Let's examine what the classifier found in your document.

In [ ]:
# Display classification results
if 'result' in locals() and result:
    result_data = result.get("result", {})
    contents = result_data.get("contents", [])
    
    print("📊 CLASSIFICATION RESULTS")
    print("=" * 50)
    print(f"\nTotal sections found: {len(contents)}")
    
    # Show summary of each classified section
    print("\n📋 Document Sections:")
    for i, content in enumerate(contents, 1):
        print(f"\n   Section {i}:")
        print(f"   • Category: {content.get('category', 'Unknown')}")
        print(f"   • Pages: {content.get('startPageNumber', '?')} - {content.get('endPageNumber', '?')}")
        
    print("\nFull result:")
    print(json.dumps(result, indent=2))
else:
    print("❌ No results available. Please run the classification step first.")

## 9. Create a Custom Analyzer (Advanced)

Now let's create a custom analyzer that can extract specific fields from documents.
This analyzer will:
- Extract common fields from loan application documents
- Generate document excerpts

In [ ]:
# Define analyzer schema with custom fields
analyzer_schema = {
    "description": "Loan application analyzer - extracts key information from loan application",
    "baseAnalyzerId": "prebuilt-documentAnalyzer",  # Built on top of the general document analyzer
    "config": {
        "returnDetails": True,
        "enableLayout": True,          # Extract layout information
        "enableBarcode": False,        # Skip barcode detection
        "enableFormula": False,        # Skip formula detection
        "estimateFieldSourceAndConfidence": True, # Set to True if you want to estimate the field location (aka grounding) and confidence
        "disableContentFiltering": False,
    },
    "fieldSchema": {
        "fields": {
            "ApplicationDate": {
                "type": "date",
                "method": "generate",
                "description": "The date when the loan application was submitted."
            },
            "ApplicantName": {
                "type": "string",
                "method": "generate",
                "description": "The full name of the loan applicant or company."
            },
            "LoanAmountRequested": {
                "type": "number",
                "method": "generate",
                "description": "The total amount of loan money requested by the applicant."
            },
            "LoanPurpose": {
                "type": "string",
                "method": "generate",
                "description": "The stated purpose or reason for the loan."
            },
            "CreditScore": {
                "type": "number",
                "method": "generate",
                "description": "The credit score of the applicant, if available."
            },
            "Summary": {
                "type": "string",
                "method": "generate",
                "description": "A brief overview of the loan application details."
            }
        }
    }
}

# Generate unique analyzer ID
analyzer_id = "analyzer-loan-application-" + str(uuid.uuid4())

# Create the analyzer
try:
    print(f"🔨 Creating custom analyzer: {analyzer_id}")
    print("\n📋 Analyzer will extract:")
    for field_name, field_info in analyzer_schema["fieldSchema"]["fields"].items():
        print(f"   • {field_name}: {field_info['description']}")
    
    response = content_understanding_client.begin_create_analyzer(analyzer_id, analyzer_schema)
    result = content_understanding_client.poll_result(response)
    
    print("\n✅ Analyzer created successfully!")
    print(f"   Analyzer ID: {analyzer_id}")
    
except Exception as e:
    print(f"\n❌ Error creating analyzer: {e}")
    analyzer_id = None  # Set to None if creation failed

## 10. Create an Enhanced Classifier with Custom Analyzer

Now we'll create a new classifier that uses the prebuilt invoice analyzer for invoices and our custom analyzer for loan application documents.
This combines classification with field extraction in one operation.

In [ ]:
# Generate unique enhanced classifier ID
enhanced_classifier_id = "classifier-enhanced-" + str(uuid.uuid4())

# Create enhanced classifier schema
enhanced_classifier_schema = {
    "categories": {
        "Loan application": {  # Both space and underscore are allowed in category names
            "description": "Documents submitted by individuals or businesses to request funding, typically including personal or business details, financial history, loan amount, purpose, and supporting documentation.",
            "analyzerId": analyzer_id  # IMPORTANT: Use created custom analyzer in previous step for loan applications
        },
        "Invoice": {
            "description": "Billing documents issued by sellers or service providers to request payment for goods or services, detailing items, prices, taxes, totals, and payment terms.",
            "analyzerId": "prebuilt-invoice"  # Use prebuilt invoice analyzer for invoices
        },
        "Bank_Statement": {  # Both space and underscore are allowed in category names
            "description": "Official statements issued by banks that summarize account activity over a period, including deposits, withdrawals, fees, and balances."
            # No analyzer specified - uses default processing
        }
    },
    "splitMode": "auto"
}

# Create the enhanced classifier
if analyzer_id:  # Only create if analyzer was successfully created
    try:
        print(f"🔨 Creating enhanced classifier: {enhanced_classifier_id}")
        print("\n📋 Configuration:")
        print("   • Loan application documents → Custom analyzer with field extraction")
        print("   • Invoice documents → Prebuilt invoice analyzer")
        print("   • Bank_Statement documents → Standard processing")
        
        response = content_understanding_client.begin_create_classifier(enhanced_classifier_id, enhanced_classifier_schema)
        result = content_understanding_client.poll_result(response)
        
        print("\n✅ Enhanced classifier created successfully!")
        
    except Exception as e:
        print(f"\n❌ Error creating enhanced classifier: {e}")
else:
    print("⚠️  Skipping enhanced classifier creation - analyzer was not created successfully.")

## 11. Process Document with Enhanced Classifier

Let's process the document again using our enhanced classifier.
Invoices and loan application documents will now have additional fields extracted.

In [ ]:
if 'enhanced_classifier_id' in locals() and analyzer_id:
    try:
        # Check if document exists
        if not file_location.exists():
            raise FileNotFoundError(f"Document not found at {file_location}")
        
        # Process with enhanced classifier
        print("📄 Processing document with enhanced classifier")
        print(f"   Document: {file_location.name}")
        print("\n⏳ Processing with classification + field extraction...")
        
        response = content_understanding_client.begin_classify(enhanced_classifier_id, file_location=str(file_location))
        enhanced_result = content_understanding_client.poll_result(response, timeout_seconds=360)
        
        print("\n✅ Enhanced processing completed!")
        
    except Exception as e:
        print(f"\n❌ Error processing document: {e}")
else:
    print("⚠️  Skipping enhanced classification - enhanced classifier was not created.")

## 12. View Enhanced Results with Extracted Fields

Let's see the classification results along with the extracted fields from loan application documents.

In [ ]:
# Display enhanced results
if 'enhanced_result' in locals() and enhanced_result:
    result_data = enhanced_result.get("result", {})
    contents = result_data.get("contents", [])
    
    print("📊 ENHANCED CLASSIFICATION RESULTS")
    print("=" * 70)
    print(f"\nTotal sections found: {len(contents)}")
    
    # Process each section
    for i, content in enumerate(contents, 1):
        print(f"\n{'='*70}")
        print(f"SECTION {i}")
        print(f"{'='*70}")
        
        category = content.get('category', 'Unknown')
        print(f"\n📁 Category: {category}")
        print(f"📄 Pages: {content.get('startPageNumber', '?')} - {content.get('endPageNumber', '?')}")
        
        # Show extracted fields from field extraction
        fields = content.get('fields', {})
        if fields:
            print("\n🔍 Extracted Information:")
            for field_name, field_data in fields.items():
                print(f"\n   {field_name}:")
                print(f"   • Value: {field_data}")
else:
    print("❌ No enhanced results available. Please run the enhanced classification step first.")

You can also see the fulll JSON result below.

In [ ]:
print(json.dumps(enhanced_result, indent=2))

## Summary and Next Steps

Congratulations! You've successfully:
1. ✅ Created a basic classifier to categorize documents
2. ✅ Created a custom analyzer to extract specific fields
3. ✅ Combined them into an enhanced classifier for intelligent document processing